### Load and inspect the file ‘ASA All NBA Raw Data.csv’, which is raw NBA data for the time range 2019-10-22 to 2022-02-27.

In [25]:
import pandas as pd
import sweetviz as sv
import numpy as np

from pathlib import Path
from zipfile import ZipFile
from io import BytesIO

In [21]:
zip_path = Path("local_data/ASA All NBA Raw Data.zip")
z = ZipFile(BytesIO(zip_path.read_bytes()))
df = pd.read_csv(z.open("ASA All NBA Raw Data.csv"))
df.head()

C:\Users\ler_s\AppData\Local\Temp\ipykernel_12352\3735818334.py:3: DtypeWarning: Columns (68,69,70) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(z.open("ASA All NBA Raw Data.csv"))


,game_id,game_date,OT,H_A,Team_Abbrev,Team_Score,Team_pace,Team_efg_pct,Team_tov_pct,Team_orb_pct,...,pf_per_minute,ts,last_60_minutes_per_game_starting,last_60_minutes_per_game_bench,PG%,SG%,SF%,PF%,C%,active_position_minutes
0,202202270CHO,2022-02-27,1,A,DET,127,96.3,0.518,10.6,39.3,...,0.051858,19.32,34.047024,18.329358,0.0,7.0,50.0,44.0,0.0,NaN
1,202202270CHO,2022-02-27,1,A,DET,127,96.3,0.518,10.6,39.3,...,0.120060,19.00,29.825758,0.000000,0.0,0.0,2.0,94.0,4.0,NaN
2,202202270CHO,2022-02-27,1,A,DET,127,96.3,0.518,10.6,39.3,...,0.122324,18.64,30.957143,0.000000,2.0,83.0,15.0,0.0,0.0,NaN
3,202202270CHO,2022-02-27,1,A,DET,127,96.3,0.518,10.6,39.3,...,0.173410,9.88,25.828472,13.904867,0.0,0.0,0.0,2.0,98.0,NaN
4,202202270CHO,2022-02-27,1,A,DET,127,96.3,0.518,10.6,39.3,...,0.090293,4.88,28.463725,25.170000,94.0,6.0,0.0,0.0,0.0,NaN


### Data Preparation

In [ ]:

# cleaning all numericfields from NA
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()

print(f"Numeric columns to clean: {num_cols}")

for col in num_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)

# cleaning DKP_per_minute column: coerce non-numeric to NaN then fill with 0
df['DKP_per_minute'] = pd.to_numeric(df['DKP_per_minute'], errors='coerce').fillna(0)


Numeric columns to clean: ['OT', 'Team_Score', 'Team_pace', 'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct', 'Team_ft_rate', 'Team_off_rtg', 'Opponent_Score', 'Opponent_pace', 'Opponent_efg_pct', 'Opponent_tov_pct', 'Opponent_orb_pct', 'Opponent_ft_rate', 'Opponent_off_rtg', 'starter', 'fg', 'fga', 'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct', 'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts', 'plus_minus', 'did_not_play', 'is_inactive', 'ts_pct', 'efg_pct', 'fg3a_per_fga_pct', 'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct', 'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'off_rtg', 'def_rtg', 'bpm', 'season', 'minutes', 'double_double', 'triple_double', 'DKP', 'FDP', 'SDP', 'DKP_per_minute', 'pf_per_minute', 'ts', 'last_60_minutes_per_game_starting', 'last_60_minutes_per_game_bench', 'PG%', 'SG%', 'SF%', 'PF%', 'C%', 'active_position_minutes']


In [24]:
report = sv.analyze(df)
report.show_html("sweetviz_eda.html")

                                             |          | [  0%]   00:00 -> (? left)

TypeError: 

Column [FDP_per_minute] has a 'mixed' inferred_type (as determined by Pandas).
This is is not currently supported; column types should not contain mixed data.
e.g. only floats or strings, but not a combination.

POSSIBLE RESOLUTIONS:
BEST -> Make sure series [FDP_per_minute] only contains a certain type of data (numerical OR string).
OR -> Convert series [FDP_per_minute] to a string (if makes sense) so it will be picked up as CATEGORICAL or TEXT.
     One way to do this is:
     df['FDP_per_minute'] = df['FDP_per_minute'].astype(str)
OR -> Convert series [FDP_per_minute] to a numerical value (if makes sense):
     One way to do this is:
     df['FDP_per_minute'] = pd.to_numeric(df['FDP_per_minute'], errors='coerce')
     # (errors='coerce' will transform string values to NaN, that can then be replaced if desired; consult Pandas manual pages for more details)


In [ ]:
report.show_html("sweetviz_eda.html")

In [ ]:
# Per-numeric-field summary: total missing (None/NaN) and total zeros
import pandas as pd

numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
total_rows = len(df)

rows = []
for c in numeric_cols:
    missing_count = int(df[c].isnull().sum())
    zero_count = int((df[c] == 0).sum())
    rows.append({
        'column': c,
        'missing_count': missing_count,
        'missing_pct': round(missing_count / total_rows * 100, 2),
        'zero_count': zero_count,
        'zero_pct': round(zero_count / total_rows * 100, 2)
    })

summary_df = pd.DataFrame(rows).sort_values(['missing_count', 'zero_count'], ascending=[False, False]).reset_index(drop=True)
print(f'Total rows: {total_rows}')
print('\nNumeric fields summary (missing and zero counts/pct):')
display(summary_df)

# Additional quick insights
n_rows_with_numeric_missing = int(df[numeric_cols].isnull().any(axis=1).sum()) if numeric_cols else 0
print(f'Rows with any numeric missing: {n_rows_with_numeric_missing} ({round(n_rows_with_numeric_missing/total_rows*100,2) if total_rows else 0}%)')

if not summary_df.empty:
    print('\nTop 5 columns by zero_count:')
    display(summary_df.sort_values('zero_count', ascending=False).head(5))

Total rows: 113125

Numeric fields summary (missing and zero counts/pct):


,column,missing_count,missing_pct,zero_count,zero_pct
0,pf_per_minute,19558,17.29,19023,16.82
1,active_position_minutes,15412,13.62,0,0.00
2,last_60_minutes_per_game_bench,2724,2.41,7340,6.49
3,last_60_minutes_per_game_starting,2724,2.41,1715,1.52
4,PG%,148,0.13,70105,61.97
...,...,...,...,...,...
65,Opponent_efg_pct,0,0.00,0,0.00
66,Opponent_tov_pct,0,0.00,0,0.00
67,Opponent_ft_rate,0,0.00,0,0.00
68,Opponent_off_rtg,0,0.00,0,0.00


Rows with any numeric missing: 32129 (28.4%)

Top 5 columns by zero_count:


,column,missing_count,missing_pct,zero_count,zero_pct
9,triple_double,0,0.0,112542,99.48
10,is_inactive,0,0.0,111781,98.81
11,OT,0,0.0,106816,94.42
12,double_double,0,0.0,105020,92.84
13,did_not_play,0,0.0,93597,82.74


In [ ]:
# NOTE: One-hot encoding is for categorical variables, not for converting non-numeric numeric fields to zero.
# Use pd.to_numeric(..., errors='coerce').fillna(0) to coerce non-numeric -> NaN -> 0 for numeric columns (already done).

import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# 1) Confirm numeric coercion (safe repeat):
numeric_cols = df.select_dtypes(include=['number']).columns.tolist()
for c in numeric_cols:
    df[c] = pd.to_numeric(df[c], errors='coerce').fillna(0)
print('Numeric coercion complete. Example types:')
print(df[numeric_cols].dtypes.to_dict())

# 2) If you WANT to one-hot encode numeric columns, do it only for a small explicit subset
# (e.g., bucketed numeric features or numeric codes treated as categories).
# Replace the list below with columns you intentionally treat as categorical:
numeric_as_cat = ['example_numeric_cat_col1', 'example_numeric_cat_col2']  # <- EDIT
numeric_as_cat = [c for c in numeric_as_cat if c in df.columns]

if numeric_as_cat:
    print('\nOne-hot encoding these numeric-as-categorical columns:', numeric_as_cat)
    df_onehot = pd.get_dummies(df, columns=numeric_as_cat, dummy_na=False, drop_first=False)
    print('Resulting shape after get_dummies (sample):', df_onehot.shape)
    display(df_onehot.head())
else:
    print('\nNo numeric-as-categorical columns specified (skipping get_dummies).')

# 3) Recommended: use ColumnTransformer in a pipeline to handle numeric and categorical separately
# Example: treat numeric_features as numeric, categorical_features with OneHotEncoder
numeric_features = df.select_dtypes(include=['number']).columns.tolist()
categorical_features = df.select_dtypes(include=['object', 'category']).columns.tolist()

cat_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
preprocessor = ColumnTransformer([
    ('num', 'passthrough', numeric_features),
    ('cat', cat_encoder, categorical_features)
])

print('\nColumnTransformer example prepared. Use in a pipeline:')
print("Pipeline([('pre', preprocessor), ('clf', YourEstimator())])")

# End of cell. If you want me to one-hot encode specific numeric columns now,
# tell me which column names to include in numeric_as_cat and I will modify the notebook accordingly.